In [7]:
# coding: utf-8
import pandas as pd
from pathlib import Path
import json
root = Path('.')
root = root.absolute()
test_cases_root = root / "testcases"
run_confs = {4: "oopsla-final-4gb", 8: "oopsla-final-8gb", 16: "oopsla-final-16gb", 32: "oopsla-final-32gb"}
results_roots = { k: root / x for k, x in run_confs.items()}

In [8]:
print("root: ", root)
print("test_cases_root: ", test_cases_root)
print("results_root: ", results_roots)
# cwd
print("cwd: ", Path.cwd())

root:  /Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether
test_cases_root:  /Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/testcases
results_root:  {4: PosixPath('/Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-4gb'), 8: PosixPath('/Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-8gb'), 16: PosixPath('/Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-16gb'), 32: PosixPath('/Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-32gb')}
cwd:  /Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether


In [25]:
import json
# Now we need to process the reports. Each report should be read, and then split into it's subjsons:
# Each json is preceded by "--------------------------------"\n and followed by "----------------------------------------------------------------"
# We can use this to split the file into a list of jsons, and then use json.loads to load each json
def split_report(report):
    with open(report, 'r') as f:
        lines = f.readlines()
    # Remove all non-json
    lines = [x for x in lines if x.startswith("{")]
    # cleanup final "-----" from the lines
    lines = [x.replace("--------------------------------", "") for x in lines]
    # load jsons
    jsons = [json.loads(x) for x in lines]
    return jsons

In [27]:
test_cases = [y for x in test_cases_root.iterdir() if x.is_dir() for y in x.iterdir() if y.suffix == ".th"]
rel_cases = [x.relative_to(test_cases_root) for x in test_cases]

# Use pathlib glob for each result root to find all .report.txt files
for r in results_roots.values():
    assert r.is_dir()
    assert r.exists()
reports = {k: {conf: [(y, split_report(y)) for y in conf.rglob("*.report.txt")] for conf in root.iterdir() if conf.is_dir()}for k, root in results_roots.items()}

In [67]:
 # Now we need to process the data from the jsons. We will need a dataframe indexed by memory (k), config, testcase (y)

import pandas
from copy import deepcopy

flattened_reports = []
for k, d in reports.items():
    if k != 4:
        continue
    for conf, data in d.items():
        fixed = []
        for (y, jsons) in data:
            if len(jsons) == 0:
                continue
            j = deepcopy(jsons[-1])
            j['key'] = (k, conf.name, y.parent.name, y.name)
            j['case'] = (y.parent.name, y.name)
            j['conf'] = conf.name.replace("case_split_proof_split_", "").replace("_stats_progress_report", "")
            j['depth'] = int(j['conf'][0])
            j['conf'] = j['conf'][1:].replace("_split_", "")
            fixed.append(j)
        flattened_reports.extend(fixed)
        
pd = pandas.json_normalize(flattened_reports).set_index('key')
pd.describe()

,apply_time,graph_size,rebuild_time,run_time,search_time,split_count,total_applied,total_graph_size,total_matches,depth,progress.CaseSplitEquivReduc,progress.CaseSplitStart,progress.CaseSplitMergeConclusions,progress.CaseSplitFindSplitters
count,4220.000000,4220.000000,4220.000000,4220.000000,4220.000000,4220.000000,4.220000e+03,4220.000000,4.220000e+03,4220.000000,258.000000,30.000000,64.000000,218.000000
mean,224.378673,10406.655687,496.787441,49281.478436,33785.510427,66.710427,2.628604e+05,10657.430569,5.499916e+05,2.500000,1.565891,1.400000,2.421875,2.334862
std,1773.964783,40338.814603,3191.641473,138262.283198,111158.582253,211.571151,1.713429e+06,40407.062421,3.433328e+06,1.118166,1.205338,0.723974,1.080927,1.153146
min,0.000000,22.000000,0.000000,19.000000,0.000000,0.000000,3.200000e+01,22.000000,8.030000e+02,1.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,37.000000,0.000000,51.000000,0.000000,0.000000,4.920000e+02,37.000000,9.125000e+03,1.750000,0.000000,1.000000,1.000000,1.000000
50%,0.000000,42.000000,0.000000,113.000000,0.000000,8.000000,2.111500e+03,42.000000,1.813400e+04,2.500000,2.000000,1.000000,2.500000,2.000000
75%,15.000000,47.000000,36.000000,2578.000000,742.000000,60.000000,6.860325e+04,47.000000,1.708790e+05,3.250000,3.000000,1.750000,3.000000,3.000000
max,87067.000000,520323.000000,71209.000000,599963.000000,595516.000000,3906.000000,8.499911e+07,520323.000000,1.489146e+08,4.000000,3.000000,3.000000,4.000000,4.000000


In [69]:
# compare split count of all columns with the same case
pivot = pd.loc[pd.split_count != 0].pivot_table(index=['case', 'depth'], columns='conf', values='split_count')
pivot.loc[pivot['colored'] != pivot['clone_keep_splits']]

conf                                      clone  clone_keep_splits  colored  \
case                               depth                                      
(clam, goal54.report.txt)          3      184.0              184.0    186.0   
                                   4      260.0              260.0    302.0   
(clam, goal55.report.txt)          4      126.0              126.0    154.0   
(clam, goal56.report.txt)          4      504.0              504.0    576.0   
(clam, goal57.report.txt)          4      114.0              114.0    146.0   
...                                         ...                ...      ...   
(leon-heap, heap-goal6.report.txt) 4      470.0              470.0    162.0   
(leon-heap, heap-goal7.report.txt) 3      274.0              274.0    284.0   
                                   4      538.0              538.0    122.0   
(leon-heap, heap-goal8.report.txt) 4      438.0              438.0    304.0   
(leon-heap, heap-goal9.report.txt) 4      312.0              312.0    164.0   

conf                                      no_cmemo  no_cremove  
case                               depth                        
(clam, goal54.report.txt)          3         296.0       186.0  
                                   4         774.0       300.0  
(clam, goal55.report.txt)          4         244.0       154.0  
(clam, goal56.report.txt)          4        1458.0       580.0  
(clam, goal57.report.txt)          4         230.0       144.0  
...                                            ...         ...  
(leon-heap, heap-goal6.report.txt) 4         702.0       162.0  
(leon-heap, heap-goal7.report.txt) 3         584.0       284.0  
                                   4         700.0       122.0  
(leon-heap, heap-goal8.report.txt) 4         506.0       304.0  
(leon-heap, heap-goal9.report.txt) 4         492.0       164.0  

[182 rows x 5 columns]

In [40]:
(4, /Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-4gb/case_split_proof_split_4_split_clone_keep_splits_stats_progress_report, /Users/eytansingher/Work/TheSy/experiments/cvc4_newth_alltogether/oopsla-final-4gb/case_split_proof_split_4_split_clone_keep_splits_stats_progress_report/leon-heap/heap-goal3.report.txt)

SyntaxError: invalid syntax (4103080003.py, line 1)